# Read files from S3 with Spark demo

## Libraries

In [ ]:
import os
import json
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import udf
from pyspark.sql.types import *
from pyspark.sql.functions import struct
from pyspark.sql.functions import countDistinct
import multiprocessing
N_CORES = min(
    multiprocessing.cpu_count(), 
    int(float(os.environ['CPU_LIMIT']))
)
print('max cores available:', N_CORES)

## Credentials and set Spark

In [ ]:
AVSLS_BUCKET = 'miba-ma-prj-options'
with open('access.json') as file:
    access_data = json.load(file)

In [ ]:
print('user:', os.environ['JUPYTERHUB_SERVICE_PREFIX'])

def uiWebUrl(self):
    from urllib.parse import urlparse
    web_url = self._jsc.sc().uiWebUrl().get()
    port = urlparse(web_url).port
    return '{}proxy/{}/jobs/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)

SparkContext.uiWebUrl = property(uiWebUrl)

conf = SparkConf()
conf.set('spark.master', 'local[*]')
conf.set('spark.executor.memory', '16G')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
spark

In [ ]:
spark._jsc.hadoopConfiguration().set('fs.s3a.access.key', access_data['aws_access_key_id'])
spark._jsc.hadoopConfiguration().set('fs.s3a.secret.key', access_data['aws_secret_access_key'])
spark._jsc.hadoopConfiguration().set('fs.s3a.impl','org.apache.hadoop.fs.s3a.S3AFileSystem')
spark._jsc.hadoopConfiguration().set('fs.s3a.multipart.size', '104857600')
spark._jsc.hadoopConfiguration().set('fs.s3a.block.size', '33554432')
spark._jsc.hadoopConfiguration().set('fs.s3a.threads.max', '256')
spark._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 'https://hb.bizmrg.com')

## List files with `boto3`

In [ ]:
session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    aws_access_key_id=access_data['aws_access_key_id'],
    aws_secret_access_key=access_data['aws_secret_access_key'],
    endpoint_url='https://hb.bizmrg.com'
)

In [ ]:
all_files = [key['Key'] for key in s3.list_objects(Bucket=AVSLS_BUCKET)['Contents']]
print('first files:', all_files[:10])

## Read a file

In [ ]:
file_path = f's3a://{AVSLS_BUCKET}/{all_files[1]}'
print('file path to load:', file_path)
sdf = spark.read.csv(
    file_path, 
    sep=',', 
    header=False
)

In [ ]:
sdf.printSchema()

In [ ]:
sdf.limit(5).toPandas()

In [ ]:
%%time
sdf.count()

In [ ]:
%%time
sdf.groupby('_c4').count().toPandas()

## Many files

In [ ]:
file_path_masked = f's3a://{AVSLS_BUCKET}/data/L3_options_2016*'
print('file path to load:', file_path_masked)
sdf = spark.read.csv(
    file_path_masked, 
    sep=',', 
    header=False
)

In [ ]:
sdf.printSchema()

In [ ]:
%%time
sdf.count()

In [ ]:
%%time
sdf.groupby('_c4').count().toPandas()